# MCMC diagnostics

In [49]:
# install.packages("coda")
# install.packages("mcmcse")
library("coda")

kappa_flag = TRUE
beta_flag = FALSE


if(beta_flag & kappa_flag){
    path_names = '../data/cv_model/priors_full_cut_phen/'
    path_diagnos = '../data/cv_model/diagnos_full_cut_phen/'
    path_data = '../data/cv_model/mcmc_final_full_cut_phen_'    
}else if (beta_flag & !kappa_flag){
    path_names = '../data/cv_model/priors_full_cut/'
    path_diagnos = '../data/cv_model/diagnos_full_cut/'
    path_data = '../data/cv_model/mcmc_final_full_cut_'    
}else if (!beta_flag & kappa_flag){
    path_names = '../data/cv_model/priors_zero_cut_phen/'
    path_diagnos = '../data/cv_model/diagnos_zero_cut_phen/'
    path_data = '../data/cv_model/mcmc_final_zero_cut_phen_'    
}else{
    path_names = '../data/cv_model/priors_zero_cut/'
    path_diagnos = '../data/cv_model/diagnos_zero_cut/'
    path_data = '../data/cv_model/mcmc_final_zero_cut_'    
}


path_data_list = c()
n_chain = 5
for(i in 1:n_chain)
{
    path_data_list = c(path_data_list, paste0(c(path_data, as.character(i-1), '/'), collapse = ''))
}


idx = 50:2000

n_cv = 20


## Get diagnostics

In [50]:
for(idata in 0:(n_cv-1))
{
    print(idata)
    x1 = read.table(paste(c(path_data_list[1], 'mcmc_5_', as.character(idata),'.txt'), collapse = ''))
    x2 = read.table(paste(c(path_data_list[2], 'mcmc_5_', as.character(idata),'.txt'), collapse = ''))
    x3 = read.table(paste(c(path_data_list[3], 'mcmc_5_', as.character(idata),'.txt'), collapse = ''))
    x4 = read.table(paste(c(path_data_list[4], 'mcmc_5_', as.character(idata),'.txt'), collapse = ''))
    x5 = read.table(paste(c(path_data_list[5], 'mcmc_5_', as.character(idata),'.txt'), collapse = ''))
    

    x1 = as.mcmc(x1[idx,])
    x2 = as.mcmc(x2[idx,])
    x3 = as.mcmc(x3[idx,])
    x4 = as.mcmc(x4[idx,])
    x5 = as.mcmc(x5[idx,])
    
    
    y = mcmc.list(x1,x2,x3)

    # Mean and std
    res = summary(y)
    d1 = res[[1]]
    
    # Mean for x_i
    d_x = cbind(summary(x1)[[1]][,'Mean'], 
                summary(x2)[[1]][,'Mean'],
               summary(x3)[[1]][,'Mean'], 
               summary(x4)[[1]][,'Mean'],
               summary(x5)[[1]][,'Mean'])
    

    # Gelman-Rubin Diagnostic
    gl = gelman.diag(y, confidence = 0.98, transform=FALSE, autoburnin=TRUE,
                multivariate=TRUE)
    Gelman_Rubin = gl[[1]][,1]

    # Effective size
    ESS = effectiveSize(y)

    # Names

    var_names = read.table(paste(c(path_names, 'names_5_', as.character(idata),'.txt'), collapse = ''))
    colnames(var_names) <- c('Variable1', 'Variable2') 


    
    # Combine all
    d = cbind(var_names, d1, Gelman_Rubin, ESS, d_x)
    head(d)

    write.table(file = paste(c(path_diagnos, 'diagnos_5_new_', as.character(idata),'.txt'), collapse = ''), 
                x = d, quote = FALSE, sep = '\t', row.names = FALSE)
   
}


[1] 0
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19


## Diagnostics summary

In [47]:
min_ess = c()
n_gl = c()
n_param = c()
mean_ess = c()
median_ess = c()
for(idata in 0:(n_cv-1))
{
    d = read.table(file = paste(c(path_diagnos, 'diagnos_5_new_', as.character(idata),'.txt'), collapse = ''), 
                   sep = '\t', header = TRUE)
    min_ess = c(min_ess, min(d[,'ESS']))
    mean_ess = c(mean_ess, mean(d[,'ESS']))
    median_ess = c(median_ess, median(d[,'ESS']))
    n_gl = c(n_gl, sum(d[,'Gelman_Rubin'] > 1.05))
    n_param = c(n_param,dim(d)[1])
}



In [48]:
n_gl
n_param
sum(n_gl) / sum(n_param)

[1] 6 3 4 0 0 3 0 1 7 0 6 0 8 2 4 0 5 5 0 1

[1] 75 77 74 78 73 76 76 76 77 71 73 77 77 75 81 75 79 77 79 80

[1] 0.03604194

In [40]:
mean(min_ess)
mean(mean_ess)
mean(median_ess)

[1] 163.5578

[1] 3383.552

[1] 3852.319